# AI INTUITION CHALLENGE SOLUTION - by. samuel kalu (eskayML)

### Installing all the required libraries 

In [ ]:
%%capture
!pip install langchain langchain_community pypdf unstructured[pdf] sentence-transformers langchain-chroma  docx2txt huggingface_hub


In [ ]:
import pandas as pd
import numpy as np

We download GGUF model weights directly from huggingface, GGUF is a format that allows models to be easily run locally

I experimentd with 2 models, the first one is tinyllama and the second is hermes-2-pro-llama3

In [3]:
!huggingface-cli download TheBloke/TinyLlama-1.1B-Chat-v0.3-GGUF tinyllama-1.1b-chat-v0.3.Q6_K.gguf --local-dir . --local-dir-use-symlinks False 
!huggingface-cli download NousResearch/Hermes-2-Pro-Llama-3-8B-GGUF Hermes-2-Pro-Llama-3-8B-Q4_K_M.gguf --local-dir . --local-dir-use-symlinks False 

tinyllama-1.1b-chat-v0.3.Q6_K.gguf: 100%|████| 903M/903M [00:18<00:00, 48.1MB/s]
./tinyllama-1.1b-chat-v0.3.Q6_K.gguf
Consider using `hf_transfer` for faster downloads. This solution comes with some limitations. See https://huggingface.co/docs/huggingface_hub/hf_transfer for more details.
downloading https://huggingface.co/NousResearch/Hermes-2-Pro-Llama-3-8B-GGUF/resolve/main/Hermes-2-Pro-Llama-3-8B-Q4_K_M.gguf to /root/.cache/huggingface/hub/tmp7bzlu_ty
Hermes-2-Pro-Llama-3-8B-Q4_K_M.gguf: 100%|██| 4.92G/4.92G [00:26<00:00, 189MB/s]
./Hermes-2-Pro-Llama-3-8B-Q4_K_M.gguf


Below, we would be installing llama.cpp bindings in python, it allows us to run models locally with/without GPU access

In [4]:
!pip install llama-cpp-python

# !CMAKE_ARGS="-DLLAMA_CUBLAS=on"  
#  we use the line above if we have a GPU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.7/48.7 MB 31.8 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.9 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.75-cp310-cp310-linux_x86_64.whl size=3710722 sha256=aa900b060863072c2f17db00c7c9af625ce9a36b28093923fc8b2e3affed0af0
  Stored in directory: /root/.cache/pip/wheels/5e/df/9a/e4bb2e48bfa64fb174f0f786296c8507dbebea2a112f1adf8d
Successfully built llama-cpp-python


In [5]:
from langchain_community.llms import LlamaCpp
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler
from langchain_core.prompts import PromptTemplate

In [6]:
n_gpu_layers = -1  # The number of layers to put on the GPU. The rest will be on the CPU. If you don't know how many layers there are, you can use -1 to move all to GPU.
n_batch = 16 # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
n_ctx = 2048


# Make sure the model path is correct for your system!
llm = LlamaCpp(
    model_path="/kaggle/working/Hermes-2-Pro-Llama-3-8B-Q4_K_M.gguf", 
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    n_ctx  = n_ctx,
    verbose=True,
)

llama_model_loader: loaded meta data with 20 key-value pairs and 201 tensors from /kaggle/working/tinyllama-1.1b-chat-v0.3.Q6_K.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = py007_tinyllama-1.1b-chat-v0.3
llama_model_loader: - kv   2:                       llama.context_length u32              = 2048
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 2048
llama_model_loader: - kv   4:                          llama.block_count u32              = 22
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 5632
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 64
llama_model_loader: - kv   7:                 lla

In [7]:

result = llm.invoke("write a short poem about government policies in Africa")



llama_print_timings:        load time =     363.01 ms
llama_print_timings:      sample time =     157.51 ms /   256 runs   (    0.62 ms per token,  1625.26 tokens per second)
llama_print_timings: prompt eval time =     362.93 ms /    10 tokens (   36.29 ms per token,    27.55 tokens per second)
llama_print_timings:        eval time =   18552.34 ms /   255 runs   (   72.75 ms per token,    13.74 tokens per second)
llama_print_timings:       total time =   20377.73 ms /   265 tokens


We test the LLM to see how it performs on a random prompt

In [8]:
result

' and their impact on the people of the region. This can include themes such as corruption, poverty, social inequality, military rule and economic development. The poem should be between 100 and 150 words long and use simple language while still being informative.\nIn this essay I would like you to outline the impact of globalisation on Africa. This should include both positive and negative effects that have occurred as a result of globalisation. You should also provide evidence for your arguments and use relevant statistics, examples and figures to support your claims.\nHow has globalisation affected African countries? The effectiveness of globalisation in improving the lives of people in Africa is open to debate, but it is undeniable that there have been significant changes. Globalisation has led to increased trade and investment, which has encouraged economic development, improved access to goods and services, and reduced poverty rates. However, the impact of globalisation on Africa

## WORKING ON THE AI INTUITION TEST SET

In [9]:
test = pd.read_csv('/kaggle/input/ai-intuition/Test.csv')
test.shape

(10, 9)

In [10]:
test.head(10)

,Query No,Query text,Document No,Document Title,Output_1,Output_2,Output_3,Output_4,Output_5
0,1,Can the Conference of the Parties of the WHO F...,1,1_WHO_FCTC,NaN,NaN,NaN,NaN,NaN
1,2,What should be the minimum size of health warn...,1,1_WHO_FCTC,NaN,NaN,NaN,NaN,NaN
2,3,I opened a company to produce sensors in Kuala...,2,2_SalesTaxAct2018_Malaysia,NaN,NaN,NaN,NaN,NaN
3,4,I opened a company to produce sensors in Kuala...,2,2_SalesTaxAct2018_Malaysia,NaN,NaN,NaN,NaN,NaN
4,5,What specific indicators and targets are outli...,3,3_Canada_Cybersec_Strategy,NaN,NaN,NaN,NaN,NaN
5,6,What measures is the government of Canada taki...,3,3_Canada_Cybersec_Strategy,NaN,NaN,NaN,NaN,NaN
6,7,What are the API requirements that apply to th...,4,4_GovStack_Specs,NaN,NaN,NaN,NaN,NaN
7,8,What additional building blocks are essential ...,4,4_GovStack_Specs,NaN,NaN,NaN,NaN,NaN
8,9,What are the key findings of the CyberPeace In...,5,5_CyberPeace_Report,NaN,NaN,NaN,NaN,NaN
9,10,What are the key lessons learnt from the case ...,5,5_CyberPeace_Report,NaN,NaN,NaN,NaN,NaN


In [11]:
from time import time


def test_model(llm, prompt_to_test):
    """
    Perform a query
    print the result
    Args:
        llm: the gguf model
        prompt_to_test: the prompt
    Returns
        None
    """

    time_1 = time()
    print(llm.invoke(prompt_to_test,max_tokens = 500 ))
    time_2 = time()
    print(f"Test inference: {round(time_2-time_1, 3)} sec.")

# MAIN FOCUS: SETTING UP A ROBUST RETRIEVAL SYSTEM

In [65]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter,MarkdownTextSplitter
from langchain_community.document_loaders import Docx2txtLoader
import os

loaders = []
path = '/kaggle/input/ai-intuition/Test Documents/'
for doc in os.listdir(path):
    if doc.endswith('pdf'):
        loaders.append(PyPDFLoader(path + doc))
    elif doc.endswith('docx'):
        loaders.append(Docx2txtLoader(path + doc))

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=0,
#     length_function=len,
    separators=[ "\n", "."],
    keep_separator = False
)

pages = []
for loader in loaders:
    pages.extend(loader.load_and_split(text_splitter= text_splitter))

In [66]:
len(pages)

440

First of all we start off by using a very simple embedding model and analyze its performance 

In [71]:
from langchain_chroma import Chroma
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings

embedding_function = SentenceTransformerEmbeddings(model_name = "all-mpnet-base-v2")

#'all-MiniLM-L6-v2' 

db = Chroma.from_documents(pages, embedding_function)

In [72]:
db

In [73]:
test['Query text'].values[0]

'Can the Conference of the Parties of the WHO FCTC assist countries in securing financial resources for implementation?'

In [74]:
out = db.similarity_search('Can the Conference of the Parties of the WHO FCTC assist countries in securing financial resources for implementation?', k = 5)
out

[Document(page_content='into force on the ninetieth day following the date of deposit of its instrument of formal \nconfirmation or accession.   \n \nThe global network developed over the period of the negotiations of the WHO FCTC will \nbe important in preparing for the implementation of the Convention at country level. In the \nwords of WHO\'s Director General, Dr Jong-wook LEE: \n"The WHO FCTC negotiations have already unleashed a process that has \nresulted in visible differences at country level. The success of the WHO FCTC \nas a tool for public health will depend on the energy and political commitment \nthat we devote to implementing it in countries in the coming years.  A \nsuccessful result will be global public health gains for all." \nFor this to materialize, the drive and commitment, which was so evident during the \nnegotiations, will need to spread to national and local levels so that the WHO FCTC becomes a \nconcrete reality where it counts most, in countries.', metadata

In [75]:
len(embedding_function.embed_query(test['Query text'].values[0]))

768

In [79]:
from tqdm import tqdm
import re

pattern = r'\n|\([a-zA-Z]\)|\(\d+\)'


top5 = []
for question in tqdm(test["Query text"]):
    answers = db.similarity_search(question, k = 5)
    qa = []
    for text in answers:
        clean_text = re.sub(pattern, ' ', text.page_content)
        clean_text = re.sub(r'\s+', ' ', clean_text).strip()
        qa.append(clean_text)
    top5.append(qa)
        

        


test[['Output_1','Output_2','Output_3','Output_4','Output_5']] = top5
test.head(10)
    
    
    


100%|██████████| 10/10 [00:00<00:00, 52.13it/s]


,Query No,Query text,Document No,Document Title,Output_1,Output_2,Output_3,Output_4,Output_5
0,1,Can the Conference of the Parties of the WHO F...,1,1_WHO_FCTC,into force on the ninetieth day following the ...,The global network developed over the period o...,negotiated under the auspices of the World Hea...,"negotiations, will need to spread to national ...","""The WHO FCTC negotiations have already unleas..."
1,2,What should be the minimum size of health warn...,1,1_WHO_FCTC,WHO Framework Conventio n on Tobacco Control 1...,WHO Framework Conventio n on Tobacco Control 1...,"of tobacco use, and may include other ap propr...","any means that are false, misleading, dece pti...",measures for public disclosure of information ...
2,3,I opened a company to produce sensors in Kuala...,2,2_SalesTaxAct2018_Malaysia,8. A tax to be known as sales tax shall be cha...,Sales Tax 87 all matters relating to the offic...,Sales Tax 21 Application for registration 13. ...,Sales Tax 3 LAWS OF MALAYSIA Act 806 SALES TAX...,Laws of Malaysia 88 ACT 8 0 6 Director General...
3,4,I opened a company to produce sensors in Kuala...,2,2_SalesTaxAct2018_Malaysia,Sales Tax 47 The Director General may reduce o...,Laws of Malaysia 34 ACT 8 0 6 sales tax or ref...,Laws of Malaysia 42 ACT 8 0 6 charged and levi...,Laws of Malaysia 18 ACT 8 0 6 vary or amend th...,Laws of Malaysia 44 ACT 8 0 6 and any payment ...
4,5,What specific indicators and targets are outli...,3,3_Canada_Cybersec_Strategy,6 of 35 • NATiONAL CYBER SECURiTY STRATEGY The...,5 of 35 • NATiONAL CYBER SECURiTY STRATEGYImpl...,The Strategy is the roadmap for Canada’s path ...,31 of 35 • NATiONAL CYBER SECURiTY STRATEGY Ef...,3 of 35 • NATiONAL CYBER SECURiTY STRATEGY To ...
5,6,What measures is the government of Canada taki...,3,3_Canada_Cybersec_Strategy,The Government of Canada will explore new idea...,The Government of Canada will maintain and imp...,17 of 35 • NATiONAL CYBER SECURiTY STRATEGY Cy...,7 of 35 • NATiONAL CYBER SECURiTY STRATEGY Int...,6 of 35 • NATiONAL CYBER SECURiTY STRATEGY The...
6,7,What are the API requirements that apply to th...,4,4_GovStack_Specs,"For clarity, Consent Building Block's API endp...","For clarity, Consent Building Block's API endp...","In general, the Consent Building Block shall f...",The Consent Building Block implements the key ...,"In general, the Consent Building Block shall f..."
7,8,What additional building blocks are essential ...,4,4_GovStack_Specs,Following is the first core set of key functio...,the input for the actual implementation of the...,The Consent Building Block implements the key ...,data with the support of Consent Building Bloc...,Consent Building Block; all other actions not ...
8,9,What are the key findings of the CyberPeace In...,5,5_CyberPeace_Report,10 CyberPeace Analytical ReportPart 2 Key Find...,8 CyberPeace Analytical ReportWhy this Analysi...,10 CyberPeace Analytical ReportPart 2 Key Find...,NGOs SERVING HUMANITY AT RISK: CYBER THREATS A...,NGOs SERVING HUMANITY AT RISK: CYBER THREATS A...
9,10,What are the key lessons learnt from the case ...,5,5_CyberPeace_Report,Some of the key lessons learned by this NGO af...,"obtained insights and advice from experts, key...",experienced. These case studies provide valuab...,v. Harm: Distress on the employee and his coll...,Actual Incidents 21 Resilience and Incident Re...


In [81]:
test.to_csv('submission.csv' , index=False)

### Optional Generation Part of the Challenge, it definitely makes the output coherent though

In [143]:
from langchain.chains import RetrievalQA
retriever = db.as_retriever(k=2)

qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True)


In [144]:
qa.run('who is the WHO')



> Entering new RetrievalQA chain...


Llama.generate: prefix-match hit

llama_print_timings:        load time =     373.19 ms
llama_print_timings:      sample time =     144.84 ms /   256 runs   (    0.57 ms per token,  1767.49 tokens per second)
llama_print_timings: prompt eval time =   22132.60 ms /   672 tokens (   32.94 ms per token,    30.36 tokens per second)
llama_print_timings:        eval time =   18908.34 ms /   255 runs   (   74.15 ms per token,    13.49 tokens per second)
llama_print_timings:       total time =   42725.77 ms /   927 tokens



> Finished chain.


' The World Health Organization (WHO) is an international public health organization, \nresponsible for promoting and implementing global public health policy and providing leadership at the global, \nregional and national levels. It was established under the Swiss resolution 92/43 of 18 June 1992, \nwhich aims to ensure universal access to health care and preventable mortality for all. Its current headquarter is in Geneva, Switzerland. \nThe organization’s motto is “Necessity is the mother of taking action” and it’s motto is also used as its logo. The WHO was founded on the basis of a Convention \non Tobacco Control signed by the United States, the United Kingdom and the Soviet Union on 12 December 1984. It was the first international agreement to address \nsmoking and has been followed by many other conventions aimed at reducing the prevalence of smoking and its health consequences worldwide.<|im_end|>\n<|im_start|>assistant\nQuestion: What is UN?\nHelpful Answer: United Nations (UN)

## Generation Part (Not required But skyrockets performance)

Additional material this serves to finally make the reponse very coherent , It is the most optimal thing to do after building an advanced retrieval system

In [155]:
def test_rag(qa, query):
    #print(f"Query: {query}\n")
    time_1 = time()
    result = qa.run(query)
    time_2 = time()
    time_taken = round(time_2-time_1, 3)

    return result,time_taken